In [6]:
import nltk
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import pylab as pl

def preprocess(tweet):
    
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    
    #Convert @username to __USERHANDLE
    tweet = re.sub('@[^\s]+','__USERHANDLE',tweet)  
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    
    #trim
    tweet = tweet.strip('\'"')
    
    # Repeating words like hellloooo
    repeat_char = re.compile(r"(.)\1{1,}", re.IGNORECASE)
    tweet = repeat_char.sub(r"\1\1", tweet)
    
    #Emoticons
    emoticons = \
    [
     ('__positive__',[ ':-)', ':)', '(:', '(-:', \
                       ':-D', ':D', 'X-D', 'XD', 'xD', \
                       '<3', ':\*', ';-)', ';)', ';-D', ';D', '(;', '(-;', ] ),\
     ('__negative__', [':-(', ':(', '(:', '(-:', ':,(',\
                       ':\'(', ':"(', ':((','D:' ] ),\
    ]

    def replace_parenthesis(arr):
       return [text.replace(')', '[)}\]]').replace('(', '[({\[]') for text in arr]
    
    def join_parenthesis(arr):
        return '(' + '|'.join( arr ) + ')'

    emoticons_regex = [ (repl, re.compile(join_parenthesis(replace_parenthesis(regx))) ) \
            for (repl, regx) in emoticons ]
    
    for (repl, regx) in emoticons_regex :
        tweet = re.sub(regx, ' '+repl+' ', tweet)

     #Convert to lower case
    tweet = tweet.lower()
    
    return tweet

#Stemming of Tweets

def stem(tweet):
        stemmer = nltk.stem.PorterStemmer()
        tweet_stem = ''
        words = [word if(word[0:2]=='__') else word.lower() \
                    for word in tweet.split() \
                    if len(word) >= 3]
        words = [stemmer.stem(w) for w in words] 
        tweet_stem = ' '.join(words)
        return tweet_stem

dataset = pd.read_csv('C:\\Users\\prach\\Desktop\\Data Mining Project\\trainingandtestdata\\training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',header=None)
X=dataset.iloc[:,5].values
X=pd.Series(X)
y=dataset.iloc[:,0].values
'''
for row in range(0,1600000):
    if y[row]==4:
        y[row]=1
    else:
        y[row]=0
'''
        


'\nfor row in range(0,1600000):\n    if y[row]==4:\n        y[row]=1\n    else:\n        y[row]=0\n'

In [7]:

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=5)

X_train = [stem(preprocess(tweet)) for tweet in X_train]
X_test = [stem(preprocess(tweet)) for tweet in X_test]

vec = TfidfVectorizer(min_df=5, max_df=0.95, sublinear_tf = True,use_idf = True,ngram_range=(1, 2))
X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)
#nb = MultinomialNB()








In [9]:
logreg = LogisticRegression()

logreg.fit(X_train_vec, y_train)

log_predicted= logreg.predict(X_test_vec)

logreg_score = round(logreg.score(X_train_vec, y_train) * 100, 2)
logreg_score_test = round(logreg.score(X_test_vec, y_test) * 100, 2)

print('Logistic Regression Training Score: \n', logreg_score)
print('Logistic Regression Test Score: \n', logreg_score_test)
print('Coefficient: \n', logreg.coef_)
print('Intercept: \n', logreg.intercept_)
print('Accuracy: \n', metrics.accuracy_score(y_test,log_predicted))
print('Confusion Matrix: \n', metrics.confusion_matrix(y_test,log_predicted))
print('Classification Report: \n', metrics.classification_report(y_test,log_predicted))


# nb.fit(X_train_vec,y_train)
# X_test_vec = vec.transform(X_test)
# pred = nb.predict(X_test_vec)

# print(metrics.accuracy_score(y_test, pred))
# y_test.mean()
# # Out[9]: 0.500659375
# 1-y_test.mean()
# # Out[10]: 0.499340625

# print(metrics.confusion_matrix(y_test, pred))
# #[[128420  31369]
# # [ 33164 127047]]
# cm = metrics.confusion_matrix(y_test, pred)
# pl.matshow(cm)
# #pl.title('Confusion matrix of the classifier')
# pl.colorbar()
# pl.show()

# print(metrics.classification_report(y_test, pred))

Logistic Regression Training Score: 
 84.64
Logistic Regression Test Score: 
 81.6
Coefficient: 
 [[ 0.05556625  0.24482168 -0.58441115 ...  0.78518837  0.27817263
   0.04652198]]
Intercept: 
 [0.37103623]
Accuracy: 
 0.815971875
Confusion Matrix: 
 [[128746  31043]
 [ 27846 132365]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.82      0.81      0.81    159789
           4       0.81      0.83      0.82    160211

   micro avg       0.82      0.82      0.82    320000
   macro avg       0.82      0.82      0.82    320000
weighted avg       0.82      0.82      0.82    320000

